In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

@njit
def diffeq_accuracy(dy, t, y):
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)
    
@njit
def diffeq_accuracy_2(t, y, dy):
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)

@njit
def diffeq_accuracy_3(t, y):
    dy = np.empty(y.shape, dtype=np.float64)
    dy[0] = np.sin(t) - y[1]  # dydt = sin(t) - x(t)
    dy[1] = np.cos(t) + y[0]  # dxdt = cos(t) + y(t)
    return dy

@njit
def correct_answer(t, c1_, c2_):
    y = np.empty((2, t.size), dtype=np.float64)
    y[0] = -c1_ * np.sin(t) + c2_ * np.cos(t) - (np.cos(t) / 2)  # -c1 * sin(t) + c2 * cos(t) - cos(t) / 2
    # At t=0; y = c2 - 1/2
    y[1] = c2_ * np.sin(t) + c1_ * np.cos(t) + (np.sin(t) / 2)   # c2 * sin(t) + c1 * cos(t) + sin(t) / 2
    # At t=0; x = c1
    return y

# Initial Conditions
# y=0 --> c2 = + 1/2
c2 = 0.5
# x=1 --> c1 = + 1
c1 = 1.0
y0 = np.asarray((0., 1.), dtype=np.float64)
time_span_ = (0., 10.)

In [2]:
from CyRK import pysolve_ivp, cyrk_ode
from scipy.integrate import solve_ivp

In [7]:
rtols = [1.0e-5, 1.0e-7, 1.0e-9, 1.0e-11]

integration_method = 'DOP853'
rk_method = 2
t_span = (0., 10.)
t_eval = np.linspace(0.0, 10.0, 100)

for rtol in rtols:
    atol = rtol/100
    
    print("\n RTOL = ", rtol)
    
    scipy_sol   = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method)
    scipy_teval = solve_ivp(diffeq_accuracy_3, t_span, y0, rtol=rtol, atol=atol, method=integration_method, t_eval=t_eval)
    
    cyrk_sol   = cyrk_ode(diffeq_accuracy_2, t_span, y0, rtol=rtol, atol=atol, rk_method=rk_method, raise_warnings=False)
    cyrk_teval = cyrk_ode(diffeq_accuracy_2, t_span, y0, rtol=rtol, atol=atol, rk_method=rk_method, t_eval=t_eval, raise_warnings=False)
    
    pysolve_sol   = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, dense_output=True, pass_dy_as_arg=True)
    pysolve_teval = pysolve_ivp(diffeq_accuracy, t_span, y0, rtol=rtol, atol=atol, method=integration_method, t_eval=t_eval, pass_dy_as_arg=True)
    
    chi_sci_sol = np.nansum((scipy_sol.y - correct_answer(scipy_sol.t, c1, c2))**2 / correct_answer(scipy_sol.t, c1, c2))
    chi_crykode_sol = np.nansum((cyrk_sol[1] - correct_answer(cyrk_sol[0], c1, c2))**2 / correct_answer(cyrk_sol[0], c1, c2))
    chi_pysolve_sol = np.nansum((pysolve_sol.y - correct_answer(pysolve_sol.t, c1, c2))**2 / correct_answer(pysolve_sol.t, c1, c2))
    
    print(f"SciPy (sol)\t|\tPySolve (sol)\t|\tcyrk_ode (sol)")
    print(f"{chi_sci_sol:0.5e}\t|\t{chi_pysolve_sol:0.5e}\t|\t{chi_crykode_sol:0.5e}")
    
    chi_sci_teval = np.nansum((scipy_teval.y - correct_answer(scipy_teval.t, c1, c2))**2 / correct_answer(scipy_teval.t, c1, c2))
    chi_crykode_teval = np.nansum((cyrk_teval[1] - correct_answer(cyrk_teval[0], c1, c2))**2 / correct_answer(cyrk_teval[0], c1, c2))
    chi_pysolve_teval = np.nansum((pysolve_teval.y - correct_answer(pysolve_teval.t, c1, c2))**2 / correct_answer(pysolve_teval.t, c1, c2))
    dense_sol = pysolve_sol(t_eval)
    chi_pysolve_dense = np.nansum((dense_sol - correct_answer(t_eval, c1, c2))**2 / correct_answer(t_eval, c1, c2))
    print()
    print(f"SciPy (teval)\t|\tPySolve (teval)\t|\tcyrk_ode (teval)\t|\tPySolve (dense)")
    print(f"{chi_sci_teval:0.5e}\t|\t{chi_pysolve_teval:0.5e}\t|\t{chi_crykode_teval:0.5e}\t|\t{chi_pysolve_dense:0.5e}")
    
    
#     fig1, ax1 = plt.subplots()
#     ax1.plot(scipy_sol.t, scipy_sol.y[0], c='b')
#     ax1.plot(scipy_sol.t, scipy_sol.y[1], c='r')
#     ax1.set(title="SciPy (sol)")
    
#     fig12, ax12 = plt.subplots()
#     ax12.plot(scipy_teval.t, scipy_teval.y[0], c='b')
#     ax12.plot(scipy_teval.t, scipy_teval.y[1], c='r')
#     ax12.set(title="SciPy (t-eval)")
    
#     fig2, ax2 = plt.subplots()
#     ax2.plot(pysolve_sol.t, pysolve_sol.y[0], c='b')
#     ax2.plot(pysolve_sol.t, pysolve_sol.y[1], c='r')
#     ax2.set(title="PySolve (sol)")
    
#     fig22, ax22 = plt.subplots()
#     ax22.plot(pysolve_teval.t, pysolve_teval.y[0], c='b')
#     ax22.plot(pysolve_teval.t, pysolve_teval.y[1], c='r')
#     ax22.set(title="PySolve (t-eval)")
    
#     fig23, ax23 = plt.subplots()
#     ax23.plot(t_eval, dense_sol[0], c='b')
#     ax23.plot(t_eval, dense_sol[1], c='r')
#     ax23.set(title="PySolve (dense)")
    
#     plt.show()
    
#     break
    
    
    


 RTOL =  1e-05
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
-5.60367e-10	|	-5.60367e-10	|	-5.60367e-10

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
5.40227e-09	|	5.40227e-09	|	-5.66794e-02	|	5.40227e-09

 RTOL =  1e-07
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
4.47365e-13	|	4.47365e-13	|	4.47363e-13

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-1.24508e-13	|	-1.24508e-13	|	-2.34177e-03	|	-1.24508e-13

 RTOL =  1e-09
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
6.11184e-18	|	6.11176e-18	|	6.11187e-18

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-1.48424e-18	|	-1.48456e-18	|	-8.29398e-04	|	-1.48456e-18

 RTOL =  1e-11
SciPy (sol)	|	PySolve (sol)	|	cyrk_ode (sol)
6.07995e-22	|	6.07929e-22	|	6.08317e-22

SciPy (teval)	|	PySolve (teval)	|	cyrk_ode (teval)	|	PySolve (dense)
-9.08363e-22	|	-9.08003e-22	|	-1.19383e-04	|	-9.08003e-22


C:\Users\joepr\AppData\Local\Temp\ipykernel_19200\2841429268.py:22: RuntimeWarning: invalid value encountered in divide
  chi_sci_sol = np.nansum((scipy_sol.y - correct_answer(scipy_sol.t, c1, c2))**2 / correct_answer(scipy_sol.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_19200\2841429268.py:23: RuntimeWarning: invalid value encountered in divide
  chi_crykode_sol = np.nansum((cyrk_sol[1] - correct_answer(cyrk_sol[0], c1, c2))**2 / correct_answer(cyrk_sol[0], c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_19200\2841429268.py:24: RuntimeWarning: invalid value encountered in divide
  chi_pysolve_sol = np.nansum((pysolve_sol.y - correct_answer(pysolve_sol.t, c1, c2))**2 / correct_answer(pysolve_sol.t, c1, c2))
C:\Users\joepr\AppData\Local\Temp\ipykernel_19200\2841429268.py:29: RuntimeWarning: invalid value encountered in divide
  chi_sci_teval = np.nansum((scipy_teval.y - correct_answer(scipy_teval.t, c1, c2))**2 / correct_answer(scipy_teval.t, c1, c2))
C:\Users\joepr\App